In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr

### OUTPUT REQUIREMENTS
* hourly/timestep precipitation in mm/h
* 24 hour average temperature in degC


"""
nco remapping
ncks -C -x -v gribfile_projection hrrr_pr_hrly_202109021300_202109040000.nc hrrr_clean.nc
ncrename -v latitude,lat -v longitude,lon hrrr_clean.nc
ncks --rgr infer --rgr scrip=hrrr_scrip.nc hrrr_clean.nc dummy.nc
rm dummy.nc

ncks --rgr infer --rgr scrip=aorc_scrip.nc /home/lt0663/Documents/hlm_forecast/AORC_pr_hrly_20210901_20210907.nc dummy.nc

ncremap -a conserve -s hrrr_scrip.nc -g aorc_scrip.nc -m weights_hrrr_to_aorc.nc
rm dummy.nc

ncremap -m weights_hrrr_to_aorc.nc -i hrrr_pr_hrly_202109021300_202109040000.nc -o hrrr_pr_regridded.nc




ds_hrrr = ds_hrrr.assign_coords(lon_b=xe.util.grid_2d(ds_hrrr.lon.min(), ds_hrrr.lon.max(), ds_hrrr.sizes['lon'])['lon_b'])
ds_hrrr = ds_hrrr.assign_coords(lat_b=xe.util.grid_2d(ds_hrrr.lat.min(), ds_hrrr.lat.max(), ds_hrrr.sizes['lat'])['lat_b'])

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

ds_hrrr_regrid = xr.open_dataset("/home/lt0663/Documents/hlm_forecast/hrrr_pr_regridded.nc")
ds_hrrr = xr.open_dataset("/home/lt0663/Documents/hlm_forecast/hrrr/hrrr_pr_hrly_202109011300_202109030000.nc")
ds_target = xr.open_dataset("/home/lt0663/Documents/hlm_forecast/AORC_pr_hrly_20210901_20210907.nc")

fig, axes = plt.subplots(1, 3, figsize=(14, 6), subplot_kw={'projection': ccrs.PlateCarree()})

proj_attrs = ds_hrrr.gribfile_projection.attrs

hrrr_proj = ccrs.LambertConformal(
    central_longitude=proj_attrs['longitude_of_central_meridian'],
    central_latitude=proj_attrs['latitude_of_projection_origin'],
    standard_parallels=proj_attrs['standard_parallel'],
    globe=ccrs.Globe(semimajor_axis=proj_attrs['semi_major_axis'],
                     semiminor_axis=proj_attrs['semi_minor_axis'])
)

# Sum over time
hrrr_total = ds_hrrr['pr'].sum(dim='time')
aorc_total = ds_target['pr'].sum(dim='time')
hrrr_regrid_total = ds_hrrr_regrid['pr'].sum(dim='time')

# Common colorbar limits
vmax = max(hrrr_total.max().values, aorc_total.max().values)

extent = [-125.0, -66.9, 24.3, 49.4] #[-81.0, -71.5, 38.5, 45.5] #[-75.055, -74.22917, 40.185, 41.02417] #[-125.0, -66.9, 24.3, 49.4] #[-81.0, -71.5, 38.5, 45.5] 

# HRRR regridded
axes[0].contourf(hrrr_total.longitude, hrrr_total.latitude, hrrr_total.values, transform=ccrs.PlateCarree(), cmap='Blues', levels=np.arange(0,vmax,50))
axes[0].add_feature(cfeature.COASTLINE)
axes[0].add_feature(cfeature.STATES, linestyle=':')
#axes[0].coastlines()
axes[0].set_title('HRRR native - Total precip')
axes[0].set_extent(extent)
#axes[0].set_global()

# AORC
axes[1].contourf(aorc_total.longitude, aorc_total.latitude, aorc_total.values, transform=ccrs.PlateCarree(), cmap='Blues', levels=np.arange(0,vmax,50))
axes[1].add_feature(cfeature.COASTLINE)
axes[1].add_feature(cfeature.STATES, linestyle=':')
axes[1].set_title('AORC - Total precip')
axes[1].set_extent(extent)
#axes[1].set_global()

cb = axes[2].contourf(hrrr_regrid_total.lon, hrrr_regrid_total.lat, hrrr_regrid_total.values, transform=ccrs.PlateCarree(), cmap='Blues', levels=np.arange(0,vmax,50))
axes[2].add_feature(cfeature.COASTLINE)
axes[2].add_feature(cfeature.STATES, linestyle=':')
axes[2].set_title('HRRR regridded - Total precip')
axes[2].set_extent(extent)
#axes[2].set_global()

fig.colorbar(cb)
plt.tight_layout()
plt.show()

# Quick stats comparison
print(f"HRRR total: min={hrrr_total.min().values:.2f}, max={hrrr_total.max().values:.2f}, mean={hrrr_total.mean().values:.2f}")
print(f"AORC total: min={aorc_total.min().values:.2f}, max={aorc_total.max().values:.2f}, mean={aorc_total.mean().values:.2f}")
"""